In [1]:
from shopping_environment import ShoppingEnvironment

## Setup Environment

In [2]:
# parameters
num_shoes        = 100

# number of each consumer type
num_average     = 900
num_abnormal    = 100
num_special     = 25
num_wealthy     = 100
num_influencers = 50
num_resellers   = 25

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.7,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

### Run Lottery

In [3]:
env.run_lottery_no_gaming()
df = env.get_consumer_df()
got_shoes = df[df['shoes_acquired'] >= 1]
got_shoes['desire'].mean()

0.480750274332349

In [4]:
env.restock(num_shoes = 100)
env.run_first_come_no_gaming()
df = env.get_consumer_df()
got_shoes = df[df['shoes_acquired'] >= 1]
got_shoes['desire'].mean()

0.8879547930223519

In [5]:
env.restock(num_shoes = 100)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
got_shoes = df[df['shoes_acquired'] >= 1]
got_shoes['desire'].mean()


0.4970509739333943